In [1]:
import pandas as pd
from IPython.display import clear_output
import time
from edc import setup_environment_variables

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_CLIENT_NAME`
* `SH_CLIENT_SECRET`
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
------


In [2]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

url = 'https://services.sentinel-hub.com/batch/v1/process/'

In [ ]:
while(True):
# Query REST endpoint for all requests
    viewtoken = 0
    active_requests = []
    while viewtoken is not None:
        response = oauth.request('GET', f'{url}?viewtoken={viewtoken}').json()
        active_requests.extend([r for r in response['member'] if r['status'] == 'PROCESSING' and r['userAction'] == 'START']) 
        viewtoken=response['view']['nextToken']

    len(active_requests)

    arids = [ar['id'] for ar in active_requests]

    clear_output(True)
    for request_id in arids:
        tiles = []
        viewtoken = 0
        while viewtoken is not None:
            response = oauth.request('GET', f'{url}{request_id}/tiles?viewtoken={viewtoken}').json()
            tiles.extend(response['member'])
            viewtoken=response['view']['nextToken']

        if 'FAILED' in [t['status'] for t in tiles]: print(tiles[0]['requestId'])
        print(pd.Series([t['status'] for t in tiles]).value_counts())

    time.sleep(1)

SCHEDULED    4
dtype: int64
SCHEDULED    4
dtype: int64
SCHEDULED    4
dtype: int64
